Parte 1 del proyecto final. Programa de simulación del modelo de COVID-19 utilizando datos de Bogotá (https://datosabiertos.bogota.gov.co/dataset/numero-de-casos-confirmados-por-el-laboratorio-de-covid-19-bogota-d-c , https://saludata.saludcapital.gov.co/osb/index.php/datos-de-salud/enfermedades-trasmisibles/ocupacion-ucis/ , https://saludata.saludcapital.gov.co/osb/index.php/datos-de-salud/ofertas-de-servicios-de-salud/ocupacioncamas/) y de México (los obtenidos en clase)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sim_COV(N,R0,IFR):
    #Iniciar arrays de compartimentos
    I0 = 1  # infectados no identificados
    I1 = 1  # identificados identificados
    S=N-I0-I1 #susceptibles
    H=0 #poblacion en el hospital
    R=0 #poblacion recuperada/removida
    D=0 #Muertos
    U=0 #Poblacion en UCI
    t=list([0]) #tiempo
    
    #Parámetros de compartimentos obtenidos de datos de Bogotá del siguiente link: 
    # https://datosabiertos.bogota.gov.co/dataset/numero-de-casos-confirmados-por-el-laboratorio-de-covid-19-bogota-d-c

    w1=0.0208 #proporcion de personas en hospital que mueren
    w3=0.9789 #Proporcion de personas en hospital que se recuperan
    w2=1-w1-w3 #Proporcion de personas en hospital que pasan a UCI
    beta=0.019 #proporcion de personas que entra a hospital y muere (por UCI o directo)
    r1=1/6.94 #tiempo promedio de actividad infecciosa de individuos de I1


    #Parámetros obtenidos de https://saludata.saludcapital.gov.co/osb/index.php/datos-de-salud/enfermedades-trasmisibles/ocupacion-ucis/
    #y de https://saludata.saludcapital.gov.co/osb/index.php/datos-de-salud/ofertas-de-servicios-de-salud/ocupacioncamas/
    #respectivamente
    cH=10798 #camas totales de hospital en Bogotá
    cU=2208 #capacidad hospitalaria para COVID en bogotá

    #Parámetros de compartimentos obtenidos de datos de México (en clase)
    
    r0=1/11 #11 días de tiempo promedio de infección
    lam = R0*r0
    
    alfa=1/11.8 #1/tiempo promedio en hospital
    mu=1/11.29 #Tiempo promedio en UCI
    theta=2/24
    p=IFR/beta
    nH=0 #conteo de personas que no tuvieron acceso a hospital
    nU=0 #conteo de personas que no tuvieron acceso a UCIs
    #matriz
    M=np.array([[S,I0,I1,R,H,D,U]])
    # Matriz de transición
    T=np.array(([-1,0,1,0,0,0,0],[-1,1,0,0,0,0,0],[0,-1,0,1,0,0,0],[0,0,-1,0,1,0,0],[0,0,0,0,-1,1,0],
                     [0,0,0,0,-1,0,1],[0,0,0,1,-1,0,0],[0,0,0,0,0,1,-1],[0,0,0,1,0,0,-1])) 
    while I0+I1>0: #Mientras haya infectados
        #Actualizar datos
        S, I0, I1, R, H, D, U = M[-1][0], M[-1][1], M[-1][2], M[-1][3], M[-1][4], M[-1][5], M[-1][6]
        #Actualizar tasas entre compartimentos
        r_S_I1=p*lam*S/N*(I0+I1) #    [-1,0,1,0,0,0,0]
        r_S_I0=(1-p)*lam*S/N*(I0+I1) # [-1,1,0,0,0,0,0]
        r_I0_R=r0*I0 # [0,-1,0,1,0,0,0]
        r_I1_H=r1*I1 # [0,0,-1,0,1,0,0]
        r_H_D=alfa*H*w1 # [0,0,0,0,-1,1,0]
        r_H_U=alfa*H*w2 # [0,0,0,0,-1,0,1]
        r_H_R=alfa*H*w3 # [0,0,0,1,-1,0,0]
        r_U_D=mu*U*theta # [0,0,0,0,0,1,-1]
        r_U_R=mu*U*(1-theta) # [0,0,0,1,0,0,-1]
        #probabilidad de cada evento
        prob_rates=np.array([r_S_I1,r_S_I0,r_I0_R,r_I1_H, r_H_D,r_H_U,r_H_R,r_U_D,r_U_R])
        tot_rate=np.sum(prob_rates)
        if tot_rate==0:
            break
        prob_rates=prob_rates/tot_rate
        # determinar siguiente tiempo
        t_next = -(np.log(np.random.random()) / tot_rate)
        t.append(t[-1]+float(t_next))
        # determinar siguiente evento
        pos=np.where(np.random.multinomial(1,prob_rates)==1)[0]
        event=T[pos]
        M=np.append(M, M[-1]+event, 0)
        #Determina si colapsa capacidad hospitalaria
        if pos==4 and M[-1][4]>=cH: #si pasa el evento 4 y el número de personas es mayor o igual a la capacidad
            nH+=1 #se suma 1 a las personas que no pudieron acceder a atencion hospitalaria por falta de capacidad
        #Determina si colapsa capacidad UCI
        if pos==6 and M[-1][6]==cU: #Análodo al anterior pero con camas de UCI
            nU+=1 #se suma 1 persona que no pudo acceder a UCI por falta de camas
    return M, np.array(t) #Devuelve la matriz con la información de los compartimentos y un array de tiempo


#Configurar el modelo a correr 
nsim, N, R0, IFR =10, 1000, 3.5, 5/1000 #Indica el número de simulaciones, tamaño de población, R0 e IFR.


plt.figure() #Crea la gráfica
for i in range(0,nsim): #Realiza el número de simulaciones indicado
    M, t =sim_COV(N, R0, IFR) #Llama la funcion de simulacion indicando el tamaño de la población, R0 e IFR.
    S, I0, I1, R, H, D, U=M[:,0],M[:,1],M[:,2],M[:,3],M[:,4],M[:,5],M[:,6] #Extrae los datos
    #Grafica todos los datos, cada uno en un color determinado
    #plt.plot(t,S,label='Susceptibles',color='grey') 
    plt.plot(t,I0,label='Infectados registrados',color='yellow') 
    plt.plot(t,I1,label='Infectados no registrados',color='blue')
    #plt.plot(t,R,label='Recuperados',color='green')
    plt.plot(t,D,label='Muertes',color='black')
    #plt.plot(t,H,label='En hospital',color='purple')
    plt.plot(t,U,label='UCI',color='red')
#Asigna la leyenda para que se muestre solo una para todas las simulaciones y se vean en una sola gráfica
handles, labels = plt.gca().get_legend_handles_labels()
labels, ids = np.unique(labels, return_index=True)
handles = [handles[i] for i in ids]
plt.legend(handles, labels, loc='best')
#Muestra la grafica resultante
plt.show()

Parte 2 
a. ¿Cuál es número de infecciones esperadas usando un modelo de urnas?
b. ¿Cuál es el número de muertes esperadas en total?
c. ¿Cómo se distribuyen las muertes por cada una de los 2 géneros y tres 
grupos de edad previamente delimitados?

In [2]:
from scipy.optimize import fsolve
import numpy as np

R0=3.5 #Definir el valor de R0
N=1000000 #Tamaño de la población
def f(s):
    return s-(1-np.exp(-R0*s))
I=fsolve(f,0.5) #hallar el número de infectados totales
print('Infectados totales esperados:', I*N)

#Para calcular el número de muertes se usa que M=I*F*theta donde F es una matriz de frecuencia poblacional
#obtenida de https://saludata.saludcapital.gov.co/osb/index.php/datos-de-salud/demografia/piramidepoblacional/
# y theta es la matriz de mortalidad ajustada obtenida en clase utilizando un IFR=5/1000
F=np.array([ [0.2185,0.222],[0.1559,0.1446],[0.1467,0.1120]])
theta=np.array([[0.002,0.002],[0.0007,0.0021],[0.0158,0.0248]])
muertes_vec=I*F*theta*10**4
print('Muertes_totales: ',np.sum(muertes_vec))
print('Distribucion_muertes por género (M,F) y edades (0-29,30-49,50+): ', muertes_vec)

Infectados totales esperados: [965984.76156382]
Muertes_totales:  61.71918137821633
Distribucion_muertes por género (M,F) y edades (0-29,30-49,50+):  [[ 4.22135341  4.28897234]
 [ 1.05417917  2.93330933]
 [22.39017439 26.83119274]]


Parte 3
Conteste las preguntas a-c anteriores, suponiendo que se aplicara una vacuna 
con una eficacia del 60% a toda la población.

In [1]:
from scipy.optimize import fsolve
import numpy as np

R0=3.5 #Definir el valor de R0
N=1000000 #Tamaño de la población
def f(s): 
    return s-(1-0.6)*(1-np.exp(-R0*s))
I=fsolve(f,0.5) #hallar el número de infectados totales
print('Infectados totales esperados:', I*N)

#Para calcular el número de muertes se usa que M=I*F*theta donde F es una matriz de frecuencia poblacional
#obtenida de https://saludata.saludcapital.gov.co/osb/index.php/datos-de-salud/demografia/piramidepoblacional/
# y theta es la matriz de mortalidad ajustada obtenida en clase utilizando un IFR=5/1000
F=np.array([ [0.2185,0.222],[0.1559,0.1446],[0.1467,0.1120]])
theta=np.array([[0.002,0.002],[0.0007,0.0021],[0.0158,0.0248]])
muertes_vec=I*F*theta*10**4
print('Muertes_totales: ',np.sum(muertes_vec))
print('Distribucion_muertes por género (M,F) y edades (0-29,30-49,50+): ', muertes_vec)

Infectados totales esperados: [204404.47153224]
Muertes_totales:  13.059912697373395
Distribucion_muertes por género (M,F) y edades (0-29,30-49,50+):  [[0.89324754 0.90755585]
 [0.2230666  0.62069462]
 [4.73780948 5.6775386 ]]
